In [66]:
! pip install rasterio 
! pip install geopandas
! pip install rioxarray

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [67]:
# Working with raster data
import rasterio
import numpy as np
from rasterio.features import rasterize
import matplotlib.pyplot as plt
import os
from sklearn import preprocessing
from shapely.geometry import Polygon
import geopandas as gpd
from shapely.geometry import box
import rioxarray
from shapely.geometry import Polygon, mapping
import rasterio.plot as plot
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
import random
import json
import joblib
import pandas as pd
import datetime
from sklearn.ensemble import RandomForestClassifier

In [68]:
shape_file = '/content/drive/MyDrive/VarunaHackathon2022/training_area/traindata.shp'
src = '/content/drive/MyDrive/VarunaHackathon2022/sentinel-2-image/'
year = '2021'

#url to the bands
b4 = src + year + '/20210210/IMG_DATA/47PQS_20210210_B04.jp2'
b8 = src + year + '/20210210/IMG_DATA/47PQS_20210210_B08.jp2'

In [69]:
outfile = routfile = r'/content/ndvi_temp.tif'

#open the bands (I can't believe how easy is this with rasterio!)
with rasterio.open(b4) as red:
    RED = red.read()
with rasterio.open(b8) as nir:
    NIR = nir.read()

#compute the ndvi
# NDVI = (NIR – Red) / (NIR + Red) NIR=B8 Red=B4
ndvi = (NIR.astype(float) - RED.astype(float)) / (NIR+RED)
#print(np.nanmin(ndvi), np.nanmax(ndvi))

profile = red.meta
profile.update(driver='GTiff')
profile.update(dtype=rasterio.float32)

with rasterio.open(outfile, 'w', **profile) as dst:
    dst.write(ndvi.astype(rasterio.float32))

ground_truth = gpd.read_file(shape_file)
NDVI_dataset = rasterio.open('/content/ndvi_temp.tif')
NDVI_dataset_p = rasterio.open('/content/ndvi_temp.tif').profile
label = gpd.read_file(shape_file).to_crs(NDVI_dataset_p['crs'])
all_mean = []
for i in range(len(ground_truth)):
    polygon_geom = mapping(ground_truth.geometry[i])

    roi_polygon_src_coords = rasterio.warp.transform_geom({'init': 'epsg:32647'},
                                                NDVI_dataset_p['crs'],                                            
                                                polygon_geom)

    out_image,out_transform = rasterio.mask.mask(NDVI_dataset,
                                        [roi_polygon_src_coords],
                                        crop=True)

    out_image = np.clip(out_image[2::-1],
                      0,2200)/2200

    #plt.figure(figsize=(10,10))
    #plot.show(out_image,
    #          transform=out_transform)
    all_mean.append(np.mean(out_image[out_image!=0]))

# JSON file
f = open (src + year + '/20210210/productInfo.json', "r")
# Reading from file
data = json.loads(f.read())
# Iterating through the json
# list
data_week = pd.Timestamp(datetime.datetime.strptime(data['timestamp'],"%Y-%m-%dT%H:%M:%S.%fZ")).week
# Closing file
f.close()
        
add = [data_week] * len(all_mean)
all_mean_t=[all_mean]
all_mean_t.append(add)
all_mean_t=np.array(all_mean_t).astype('f')
all_mean_t = np.nan_to_num(all_mean_t, nan=0)
all_mean_t = np.transpose(all_mean_t)

df_test = pd.DataFrame(all_mean_t, columns=['mean', 'week'])

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:11: RuntimeWarning: invalid value encountered in true_divide
  # This is added back by InteractiveShellApp.init_path()
/usr/local/lib/python3.7/dist-packages/numpy/core/fromnumeric.py:3441: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/usr/local/lib/python3.7/dist-packages/numpy/core/_methods.py:189: RuntimeWarning: invalid value encountered in true_divide
  ret = ret.dtype.type(ret / rcount)


In [149]:
clf = joblib.load("/content/drive/MyDrive/ARV_Hackathon/Model/random_forest_full.joblib")

In [150]:
ans = clf.predict(df_test)

In [151]:
send = pd.DataFrame(ans, columns=['crop_type'])
send.to_csv("./final.csv")

In [152]:
ground_truth = gpd.read_file('/content/drive/MyDrive/VarunaHackathon2022/training_area/traindata.shp')

In [153]:
accuracy_score(ans, np.array(ground_truth['crop_type']).astype(int))

0.5140470766894457

In [154]:
confusion_matrix(ans, np.array(ground_truth['crop_type']).astype(int))

array([[207,  11,  41,   5],
       [142, 166, 109,  30],
       [ 97,  41, 222,  17],
       [ 64,  22,  61,  82]])